In [1]:
import ray

# Start the ray cluster and go checkout the dashboard
- CPUs
- GPUs
- local_mode

- 2024-10-16 22:53:05,514 - INFO - Note: detected 80 virtual cores but 
- NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
- 2024-10-16 22:53:05,515 - INFO - Note: NumExpr detected 80 cores but - "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
- 2024-10-16 22:53:05,515 - INFO - NumExpr defaulting to 8 threads.
Stopped all 6 Ray processes.

In [2]:
# !ray stop
# ray.shutdown()

In [3]:
ray.init(num_cpus=10)

2024-10-16 23:01:01,257	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8265


In [6]:
# Python fxn decorated with ray remote
@ray.remote
def f(x):
    for burn_cycles in range(2000000): # burn more cycles to see the cluster
        x += burn_cycles
    return x**0.5


In [7]:
futures = [f.remote(i) for i in range(400)]
print(ray.get(futures))

[1414213.2088196604, 1414213.2088200138, 1414213.2088203675, 1414213.208820721, 1414213.2088210746, 1414213.208821428, 1414213.2088217817, 1414213.208822135, 1414213.2088224888, 1414213.2088228422, 1414213.208823196, 1414213.2088235493, 1414213.208823903, 1414213.2088242564, 1414213.20882461, 1414213.2088249635, 1414213.2088253172, 1414213.2088256706, 1414213.2088260243, 1414213.2088263778, 1414213.2088267314, 1414213.2088270849, 1414213.2088274385, 1414213.208827792, 1414213.2088281456, 1414213.208828499, 1414213.2088288527, 1414213.2088292062, 1414213.2088295599, 1414213.2088299133, 1414213.208830267, 1414213.2088306204, 1414213.208830974, 1414213.2088313275, 1414213.2088316812, 1414213.2088320346, 1414213.2088323883, 1414213.2088327417, 1414213.2088330954, 1414213.2088334488, 1414213.2088338025, 1414213.208834156, 1414213.2088345096, 1414213.208834863, 1414213.2088352167, 1414213.2088355701, 1414213.2088359238, 1414213.2088362772, 1414213.208836631, 1414213.2088369844, 1414213.20883

In [9]:
ray.shutdown()
ray.init(num_cpus=5)

2024-10-16 23:11:04,470	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8265


In [10]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.n = 0

    def increment(self):
        self.n +=1
    
    def read(self):
        return self.n

In [12]:
# setup N actors
# Here we are creating multiple objects of Counter class remotely on ray cluster
counters = [Counter.remote() for i in range(20)]

# call actions on the Actors
# We are calling the increment function on all created objects parallely in ray cluster with remote ports
[c.increment.remote() for c in counters]

# retrieve data from the actors 
# Read back the state of the object
# We are retrieving the information from all objects as well
futures = [c.read.remote() for c in counters]
print(ray.get(futures))

(raylet) WARNING: 32 PYTHON worker processes have been started on node: 064299e57e79dbd4fa5f23243dc5e292362f0e825fb580d6b8ac9d27 with address: 10.56.7.46. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
(raylet) WARNING: 36 PYTHON worker processes have been started on node: 064299e57e79dbd4fa5f23243dc5e292362f0e825fb580d6b8ac9d27 with address: 10.56.7.46. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
ray.shutdown()